In [14]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision training
set_global_policy('mixed_float16')

# Path to your dataset
dataset_dir = "C:/Users/Ashwin/Desktop/Plant Disease prediction/Plant_leave_diseases_dataset_with_augmentation"  # Replace with the path to your dataset folder


In [16]:
# Data Augmentation and Generators
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Reserve 20% of data for validation
)

train_generator = data_gen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),  # Reduced image size
    batch_size=16,          # Reduced batch size
    class_mode="categorical",
    subset="training"  # Use 80% of data for training
)

val_generator = data_gen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),  # Reduced image size
    batch_size=16,          # Reduced batch size
    class_mode="categorical",
    subset="validation"  # Use 20% of data for validation
)

Found 49197 images belonging to 39 classes.
Found 12289 images belonging to 39 classes.


In [17]:
# Load a pre-trained model (MobileNetV2) and fine-tune
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers
global_avg_pooling = GlobalAveragePooling2D()(base_model.output)
dropout_layer = Dropout(0.5)(global_avg_pooling)
output_layer = Dense(len(train_generator.class_indices), activation="softmax")(dropout_layer)

model = Model(inputs=base_model.input, outputs=output_layer)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [18]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.keras", monitor="val_accuracy", save_best_only=True)


In [19]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Reduced epochs
    callbacks=[early_stopping, model_checkpoint]
)

C:\Users\Ashwin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 1049s 340ms/step - accuracy: 0.6106 - loss: 1.4731 - val_accuracy: 0.8577 - val_loss: 0.4603
Epoch 2/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 871s 283ms/step - accuracy: 0.8003 - loss: 0.6609 - val_accuracy: 0.8701 - val_loss: 0.4103
Epoch 3/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 867s 282ms/step - accuracy: 0.8089 - loss: 0.6408 - val_accuracy: 0.8728 - val_loss: 0.3989
Epoch 4/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 867s 282ms/step - accuracy: 0.8180 - loss: 0.6349 - val_accuracy: 0.8849 - val_loss: 0.3806
Epoch 5/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 867s 282ms/step - accuracy: 0.8193 - loss: 0.6263 - val_accuracy: 0.8731 - val_loss: 0.4052
Epoch 6/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 867s 282ms/step - accuracy: 0.8252 - loss: 0.6163 - val_accuracy: 0.8752 - val_loss: 0.4128
Epoch 7/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 866s 282ms/step - accuracy: 0.8244 - loss: 0.6219 - val_accuracy: 0.8884 - val_loss: 0.3696
Epoch 8/10
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 868s 282ms/step - a

In [20]:
# Fine-tune the model
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,  # Fewer epochs for fine-tuning
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/5
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 2279s 739ms/step - accuracy: 0.4844 - loss: 4.7112 - val_accuracy: 0.8753 - val_loss: 0.4701
Epoch 2/5
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 2276s 740ms/step - accuracy: 0.7894 - loss: 0.8293 - val_accuracy: 0.9007 - val_loss: 0.3281
Epoch 3/5
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 2276s 740ms/step - accuracy: 0.8221 - loss: 0.6014 - val_accuracy: 0.9089 - val_loss: 0.2796
Epoch 4/5
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 2244s 730ms/step - accuracy: 0.8461 - loss: 0.4956 - val_accuracy: 0.9212 - val_loss: 0.2446
Epoch 5/5
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 2247s 731ms/step - accuracy: 0.8642 - loss: 0.4313 - val_accuracy: 0.9212 - val_loss: 0.2352


In [21]:
# Save the final model
model.save("plant_disease_model.keras")

# Evaluate the model
final_loss, final_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {final_accuracy * 100:.2f}%")



769/769 ━━━━━━━━━━━━━━━━━━━━ 184s 239ms/step - accuracy: 0.9255 - loss: 0.2253
Validation Accuracy: 92.77%


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Load the saved model
model_path = "C:/Users/Ashwin/Desktop/Plant Disease prediction/plant_disease_model.keras"  # Update this with the path to your model file
model = tf.keras.models.load_model(model_path)

# Path to the test dataset directory
test_dataset_dir = "C:/Users/Ashwin/Desktop/Plant Disease prediction/Plant_leave_diseases_dataset_with_augmentation"  # Update this with the path to your test dataset

# Data Generator for Test Set
test_data_gen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_data_gen.flow_from_directory(
    test_dataset_dir,
    target_size=(128, 128),  # Same size as during training
    batch_size=16,          # Adjust batch size as needed
    class_mode="categorical",
    shuffle=False           # Do not shuffle to align predictions with filenames
)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Evaluate the model
print("Evaluating the model on the test set...")
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict on the test set
print("Generating predictions...")
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)


C:\Users\Ashwin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 48 variables whereas the saved optimizer has 52 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Found 61486 images belonging to 39 classes.
Evaluating the model on the test set...


C:\Users\Ashwin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3843/3843 ━━━━━━━━━━━━━━━━━━━━ 863s 224ms/step - accuracy: 0.9448 - loss: 0.1754
Test Loss: 0.22280895709991455
Test Accuracy: 0.9271866679191589
Generating predictions...
3843/3843 ━━━━━━━━━━━━━━━━━━━━ 815s 212ms/step

Classification Report:
                                               precision    recall  f1-score   support

                           Apple___Apple_scab       0.92      0.87      0.89      1000
                            Apple___Black_rot       0.97      0.94      0.95      1000
                     Apple___Cedar_apple_rust       0.98      0.99      0.99      1000
                              Apple___healthy       0.93      0.94      0.94      1645
                    Background_without_leaves       0.99      0.98      0.98      1143
                          Blueberry___healthy       0.99      0.97      0.98      1502
                      Cherry___Powdery_mildew       1.00      0.96      0.98      1052
                             Cherry___healthy       0.71    

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model
model_path = "C:/Users/Ashwin/Desktop/Plant Disease prediction/plant_disease_model.keras"  # Update with the path to your model file
model = tf.keras.models.load_model(model_path)

# Class labels (replace with the labels in your dataset)
class_labels = [
    "Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust",  
    "Apple___healthy", "Background_without_leaves", "Blueberry___healthy", 
    "Cherry___healthy", "Cherry___Powdery_mildew", "Corn___Cercospora_leaf_spot Gray_leaf_spot", 
    "Corn___Common_rust", "Corn___healthy", "Corn___Northern_Leaf_Blight", "Grape___Black_rot",
    "Grape___Esca_(Black_Measles)", "Grape___healthy", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Orange___Haunglongbing_(Citrus_greening)", "Peach___Bacterial_spot", "Peach___healthy", "Pepper_bell___Bacterial_spot",
    "Pepper_bell___healthy", "Potato___Early_blight", "Potato___healthy", "Potato___Late_blight",
    "Raspberry___healthy", "Soybean___healthy", "Squash___Powdery_mildew", "Strawberry___healthy",
    "Strawberry___Leaf_scorch", "Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___healthy",
    "Tomato___Late_blight", "Tomato___Leaf_Mold", "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot", "Tomato___Tomato_mosaic_virus", "Tomato___Tomato_Yellow_Leaf_Curl_Virus"

]

# Function to predict the disease from an image
def predict_disease(image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(128, 128))  # Ensure the size matches the training images
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Make a prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    # Get the disease label
    predicted_label = class_labels[predicted_class]
    confidence = predictions[0][predicted_class] * 100
    
    print(f"Predicted Disease: {predicted_label}")
    print(f"Confidence: {confidence:.2f}%")
    return predicted_label, confidence

# Example usage
image_path = "Plant_leave_diseases_dataset_with_augmentation/Corn___Northern_Leaf_Blight\image (12).JPG"  # Replace with the path to the plant image
predict_disease(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Disease: Corn___healthy
Confidence: 99.88%


('Corn___healthy', np.float16(99.9))